In [ ]:
!pip install neo4j
!pip install py2neo

# Install required packages
!pip install pandas neo4j python-dotenv matplotlib seaborn

In [ ]:

# Import libraries
import pandas as pd
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os


In [ ]:
# Upload your Excel file
from google.colab import files
uploaded = files.upload()

# Load the data
file_name = list(uploaded.keys())[0]
df = pd.read_excel(file_name)

In [ ]:
# Display basic information
print("Data shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nData types:\n", df.dtypes)
print("\nMissing values:\n", df.isnull().sum())

In [ ]:

# Handle missing values
df['Debit Amount'] = df['Debit Amount'].fillna(0)
df['Credit Amount'] = df['Credit Amount'].fillna(0)
df['Description'] = df['Description'].fillna('No Description')

# Basic analysis
print("\n=== Transaction Balance Check ===")
voucher_balances = df.groupby('Voucher No').apply(
    lambda x: abs(x['Debit Amount'].sum() - x['Credit Amount'].sum())
)
unbalanced = voucher_balances[voucher_balances > 0.01]
print(f"Unbalanced transactions: {len(unbalanced)}")

print("\n=== Account Activity Summary ===")
account_activity = df.groupby(['Account Name', 'Class Name']).agg({
    'Debit Amount': 'sum',
    'Credit Amount': 'sum'
}).reset_index()
account_activity['Net Flow'] = account_activity['Debit Amount'] - account_activity['Credit Amount']
print(account_activity.sort_values('Debit Amount', ascending=False).head(10))

# Visualize monthly activity
df['Month'] = df['Transaction Date'].dt.to_period('M')
monthly_activity = df.groupby('Month').agg({
    'Debit Amount': 'sum',
    'Credit Amount': 'sum'
}).reset_index()

plt.figure(figsize=(12, 6))
monthly_activity['Month'] = monthly_activity['Month'].astype(str)
plt.plot(monthly_activity['Month'], monthly_activity['Debit Amount'], label='Total Debits')
plt.plot(monthly_activity['Month'], monthly_activity['Credit Amount'], label='Total Credits')
plt.title('Monthly Financial Activity')
plt.xlabel('Month')
plt.ylabel('Amount')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# X axis positions
x = np.arange(len(monthly_activity['Month']))
width = 0.35  # bar width

# Plot side-by-side bars
plt.figure(figsize=(12,6))
plt.bar(x - width/2, monthly_activity['Debit Amount'], width, label='Total Debits')
plt.bar(x + width/2, monthly_activity['Credit Amount'], width, label='Total Credits')

# Labels and formatting
plt.xticks(x, monthly_activity['Month'], rotation=45)
plt.title('Monthly Financial Activity (Debits vs Credits)')
plt.xlabel('Month')
plt.ylabel('Amount')
plt.legend()
plt.tight_layout()
plt.show()

monthly_activity['Difference'] = monthly_activity['Debit Amount'] - monthly_activity['Credit Amount']
print(monthly_activity[['Month','Debit Amount','Credit Amount','Difference']])

plt.figure(figsize=(12,6))
plt.bar(monthly_activity['Month'], monthly_activity['Difference'])
plt.title('Monthly Debit - Credit Difference')
plt.xlabel('Month')
plt.ylabel('Net Amount')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:


from google.colab import files
import re
from neo4j import GraphDatabase

# Upload your AuraDB credentials file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Read and parse the credentials
with open(file_name, "r") as f:
    content = f.read()

uri = re.search(r"NEO4J_URI=(.*)", content).group(1).strip()
user = re.search(r"NEO4J_USERNAME=(.*)", content).group(1).strip()
password = re.search(r"NEO4J_PASSWORD=(.*)", content).group(1).strip()

# Test the connection
try:
    driver = GraphDatabase.driver(uri, auth=(user, password))
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful' AS result")
        print(result.single()["result"])
    driver.close()
except Exception as e:
    print(f"Connection failed: {e}")

In [ ]:

from neo4j import GraphDatabase
import numpy as np

class Neo4jKGManager:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self.driver.close()

    def create_constraints(self):
        """Create constraints for data integrity"""
        with self.driver.session() as session:
            queries = [
                "CREATE CONSTRAINT account_code_unique IF NOT EXISTS FOR (a:Account) REQUIRE a.code IS UNIQUE",
                "CREATE CONSTRAINT transaction_voucher_unique IF NOT EXISTS FOR (t:Transaction) REQUIRE t.voucher_no IS UNIQUE",
                "CREATE CONSTRAINT amount_id_unique IF NOT EXISTS FOR (amt:Amount) REQUIRE amt.id IS UNIQUE"
            ]
            for query in queries:
                try:
                    session.run(query)
                    print(f"Executed: {query}")
                except Exception as e:
                    print(f"Error executing {query}: {e}")

    def import_data(self, df, batch_size=1000):
        """Import data into Neo4j in batches"""
        total_batches = (len(df) // batch_size) + 1
        for i in range(0, len(df), batch_size):
            batch = df[i:i+batch_size]
            self._import_batch(batch)
            print(f"Processed batch {i//batch_size + 1}/{total_batches}")

        # After raw load, enrich with account metrics
        self._enrich_accounts(df)

    def _import_batch(self, batch):
        """Import a single batch of data"""
        with self.driver.session() as session:
            for _, row in batch.iterrows():
                # Create or update account
                account_query = """
                MERGE (acc:Account {code: $code})
                SET acc.name = $name,
                    acc.class = $class
                """
                session.run(account_query, {
                    'code': str(row['Account Code']),
                    'name': row['Account Name'],
                    'class': row['Class Name']
                })

                # Create or update transaction
                transaction_query = """
                MERGE (t:Transaction {voucher_no: $voucher_no})
                SET t.date = date($date),
                    t.description = $description
                """
                session.run(transaction_query, {
                    'voucher_no': row['Voucher No'],
                    'date': row['Transaction Date'].strftime('%Y-%m-%d'),
                    'description': row['Description']
                })

                # Create debit amount if exists
                if row['Debit Amount'] > 0:
                    debit_query = """
                    MATCH (acc:Account {code: $acc_code})
                    MATCH (t:Transaction {voucher_no: $voucher_no})
                    MERGE (amt:Amount {id: $amt_id})
                    SET amt.type = 'Debit',
                        amt.value = $value
                    MERGE (t)-[:HAS_DEBIT]->(amt)
                    MERGE (amt)-[:TO]->(acc)
                    """
                    session.run(debit_query, {
                        'acc_code': str(row['Account Code']),
                        'voucher_no': row['Voucher No'],
                        'amt_id': f"{row['Voucher No']}-D-{row['Account Code']}",
                        'value': float(row['Debit Amount'])
                    })

                # Create credit amount if exists
                if row['Credit Amount'] > 0:
                    credit_query = """
                    MATCH (acc:Account {code: $acc_code})
                    MATCH (t:Transaction {voucher_no: $voucher_no})
                    MERGE (amt:Amount {id: $amt_id})
                    SET amt.type = 'Credit',
                        amt.value = $value
                    MERGE (t)-[:HAS_CREDIT]->(amt)
                    MERGE (amt)-[:FROM]->(acc)
                    """
                    session.run(credit_query, {
                        'acc_code': str(row['Account Code']),
                        'voucher_no': row['Voucher No'],
                        'amt_id': f"{row['Voucher No']}-C-{row['Account Code']}",
                        'value': float(row['Credit Amount'])
                    })

    def _enrich_accounts(self, df):
        """Attach analytics metrics to Account nodes"""
        # Compute account-level aggregates
        account_summary = df.groupby(['Account Code','Account Name','Class Name']).agg({
            'Debit Amount':'sum',
            'Credit Amount':'sum'
        }).reset_index()
        account_summary['Net Flow'] = account_summary['Debit Amount'] - account_summary['Credit Amount']
        account_summary['is_balanced'] = (np.isclose(account_summary['Net Flow'], 0))

        # Volatility (std dev) per account
        account_std = df.groupby('Account Code')[['Debit Amount','Credit Amount']].std().reset_index()
        account_summary = account_summary.merge(account_std, on='Account Code', how='left')

        # Push into Neo4j
        with self.driver.session() as session:
            for _, row in account_summary.iterrows():
                query = """
                MATCH (acc:Account {code: $code})
                SET acc.total_debit = $total_debit,
                    acc.total_credit = $total_credit,
                    acc.net_flow = $net_flow,
                    acc.is_balanced = $is_balanced,
                    acc.debit_stddev = $debit_stddev,
                    acc.credit_stddev = $credit_stddev
                """
                session.run(query, {
                    'code': str(row['Account Code']),
                    'total_debit': float(row['Debit Amount']),
                    'total_credit': float(row['Credit Amount']),
                    'net_flow': float(row['Net Flow']),
                    'is_balanced': bool(row['is_balanced']),
                    'debit_stddev': float(row.get('Debit Amount_y', 0) if not np.isnan(row.get('Debit Amount_y', 0)) else 0),
                    'credit_stddev': float(row.get('Credit Amount_y', 0) if not np.isnan(row.get('Credit Amount_y', 0)) else 0),
                })
        print("Account enrichment completed.")


##OR WE CAN DO LIKE THIS

In [ ]:

# --- Step 1: Upload your Neo4j AuraDB credentials file ---
from google.colab import files
import re
import pandas as pd
from neo4j import GraphDatabase

uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Parse credentials
with open(file_name, "r") as f:
    content = f.read()

uri = re.search(r"NEO4J_URI=(.*)", content).group(1).strip()
user = re.search(r"NEO4J_USERNAME=(.*)", content).group(1).strip()
password = re.search(r"NEO4J_PASSWORD=(.*)", content).group(1).strip()

print("Credentials loaded successfully.")



In [ ]:
# --- Step 2: Define KG Manager with fast import ---
class Neo4jKGManager:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self.driver.close()

    def create_constraints(self):
        with self.driver.session() as session:
            queries = [
                "CREATE CONSTRAINT account_code_unique IF NOT EXISTS FOR (a:Account) REQUIRE a.code IS UNIQUE",
                "CREATE CONSTRAINT transaction_voucher_unique IF NOT EXISTS FOR (t:Transaction) REQUIRE t.voucher_no IS UNIQUE",
                "CREATE CONSTRAINT amount_id_unique IF NOT EXISTS FOR (amt:Amount) REQUIRE amt.id IS UNIQUE",
                "CREATE CONSTRAINT month_value_unique IF NOT EXISTS FOR (m:Month) REQUIRE m.value IS UNIQUE"
            ]
            for query in queries:
                session.run(query)
                print(f"Executed: {query}")

    def import_data(self, df, batch_size=2000):
        total_batches = (len(df) // batch_size) + 1

        for i in range(0, len(df), batch_size):
            batch = df.iloc[i:i+batch_size].copy()

            # Ensure proper date formatting
            batch['Transaction Date'] = pd.to_datetime(batch['Transaction Date'], errors="coerce")
            batch['Transaction Date'] = batch['Transaction Date'].dt.strftime('%Y-%m-%d')

            # Add Month column
            batch['Month'] = pd.to_datetime(batch['Transaction Date'], errors="coerce").dt.to_period('M').astype(str)

            # Convert Account Code to string
            batch['Account Code'] = batch['Account Code'].astype(str)

            # Convert to dict
            records = batch.to_dict("records")

            query = """
            UNWIND $rows AS row
            MERGE (acc:Account {code: row.`Account Code`})
              SET acc.name = row.`Account Name`,
                  acc.class = row.`Class Name`

            MERGE (t:Transaction {voucher_no: row.`Voucher No`})
              SET t.date = date(row.`Transaction Date`),
                  t.description = row.Description

            MERGE (m:Month {value: row.Month})
            MERGE (t)-[:OCCURRED_IN]->(m)

            FOREACH (_ IN CASE WHEN row.`Debit Amount` > 0 THEN [1] ELSE [] END |
                MERGE (amt:Amount {id: row.`Voucher No` + "-D-" + row.`Account Code`})
                  SET amt.type = 'Debit', amt.value = row.`Debit Amount`
                MERGE (t)-[:HAS_DEBIT]->(amt)
                MERGE (amt)-[:TO]->(acc)
            )

            FOREACH (_ IN CASE WHEN row.`Credit Amount` > 0 THEN [1] ELSE [] END |
                MERGE (amt:Amount {id: row.`Voucher No` + "-C-" + row.`Account Code`})
                  SET amt.type = 'Credit', amt.value = row.`Credit Amount`
                MERGE (t)-[:HAS_CREDIT]->(amt)
                MERGE (amt)-[:FROM]->(acc)
            )
            """
            with self.driver.session() as session:
                session.run(query, rows=records)

            print(f"Processed batch {i//batch_size + 1}/{total_batches}")

In [ ]:


# --- Step 3: Run constraints + import ---
kg_manager = Neo4jKGManager(uri, user, password)

try:
    print("Creating constraints...")
    kg_manager.create_constraints()

    print("Importing data...")
    kg_manager.import_data(df, batch_size=2000)

    print("Data import completed successfully!")

except Exception as e:
    print(f"Error: {e}")
finally:
    kg_manager.close()

